# Reconocimiento de Dígitos con CNN y MNIST

Este notebook implementa un sistema completo de reconocimiento de dígitos manuscritos utilizando redes neuronales convolucionales (CNN) con el dataset MNIST. Se incluye:
- Análisis exploratorio y visualización de datos
- Preprocesamiento de imágenes
- Entrenamiento de un modelo base y de un modelo mejorado
- Visualización de curvas de entrenamiento
- Matriz de confusión y análisis de errores
- Guardado del modelo
- Celda interactiva para probar el modelo con una imagen PNG

El notebook está orientado a la ejecución en entorno local.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import os

In [ ]:
# Cargar dataset MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalizar imágenes a rango [0,1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Redimensionar a (28,28,1)
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# Convertir etiquetas a one-hot
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

# Dividir el set de entrenamiento en entrenamiento y validación
# Usamos aproximadamente 50k para entrenamiento y 10k para validación (de los 60k originales)
x_train_final, x_val, y_train_final, y_val = train_test_split(x_train, y_train_cat, test_size=0.166, stratify=y_train)

In [ ]:
# Visualización: Mostrar 10 imágenes por cada dígito (0-9)
fig, axes = plt.subplots(10, 10, figsize=(10, 10))
for digit in range(10):
    # Encontrar índices de ejemplos de cada dígito
    idx = np.where(y_train == digit)[0][:10]
    for j, i in enumerate(idx):
        axes[digit, j].imshow(x_train[i].reshape(28,28), cmap='gray')
        axes[digit, j].axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Modelo Base: CNN sencilla
model_base = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model_base.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar modelo base
history_base = model_base.fit(x_train_final, y_train_final, epochs=10, batch_size=64, validation_data=(x_val, y_val))

In [ ]:
# Evaluar el modelo base en el conjunto de prueba
test_loss_base, test_acc_base = model_base.evaluate(x_test, y_test_cat)
print(f"Precisión en test del modelo base: {test_acc_base:.4f}")

In [ ]:
# Modelo Mejorado: Agregamos una capa convolucional extra
model_mejorado = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model_mejorado.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar modelo mejorado
history_mejorado = model_mejorado.fit(x_train_final, y_train_final, epochs=10, batch_size=64, validation_data=(x_val, y_val))

In [ ]:
# Evaluar el modelo mejorado en el conjunto de prueba
test_loss_mej, test_acc_mej = model_mejorado.evaluate(x_test, y_test_cat)
print(f"Precisión en test del modelo mejorado: {test_acc_mej:.4f}")

In [ ]:
# Gráficas de entrenamiento: Comparativa de curvas de exactitud y pérdida
plt.figure(figsize=(14,5))

# Curva de Exactitud
plt.subplot(1,2,1)
plt.plot(history_base.history['accuracy'], label='Train Base')
plt.plot(history_base.history['val_accuracy'], label='Val Base')
plt.plot(history_mejorado.history['accuracy'], label='Train Mejorado')
plt.plot(history_mejorado.history['val_accuracy'], label='Val Mejorado')
plt.title('Curva de Exactitud')
plt.xlabel('Época')
plt.ylabel('Exactitud')
plt.legend()

# Curva de Pérdida
plt.subplot(1,2,2)
plt.plot(history_base.history['loss'], label='Train Base')
plt.plot(history_base.history['val_loss'], label='Val Base')
plt.plot(history_mejorado.history['loss'], label='Train Mejorado')
plt.plot(history_mejorado.history['val_loss'], label='Val Mejorado')
plt.title('Curva de Pérdida')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.legend()

plt.show()

In [ ]:
# Matriz de Confusión para el modelo mejorado
y_pred = model_mejorado.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
conf_mtx = confusion_matrix(y_test, y_pred_classes)

plt.figure(figsize=(10,8))
sns.heatmap(conf_mtx, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Etiqueta Predicha')
plt.ylabel('Etiqueta Verdadera')
plt.title('Matriz de Confusión del Modelo Mejorado')
plt.show()

In [ ]:
# Guardar el modelo mejorado
model_mejorado.save("mnist_cnn_model.h5")
print("Modelo guardado como mnist_cnn_model.h5")

In [ ]:
# Celda para probar el modelo con una imagen PNG
from PIL import Image

# Cambia la siguiente ruta a la ubicación de tu imagen PNG
image_path = 'ruta_a_tu_imagen.png'  

try:
    img = Image.open(image_path).convert('L').resize((28,28))
except Exception as e:
    print("Error al abrir la imagen. Asegúrate de que la ruta es correcta.")
else:
    # Mostrar la imagen
    plt.figure(figsize=(3,3))
    plt.imshow(img, cmap='gray')
    plt.title("Imagen de entrada")
    plt.axis("off")
    plt.show()

    # Preprocesar la imagen
    img_array = np.array(img, dtype='float32') / 255.0
    img_array = img_array.reshape((1, 28, 28, 1))

    # Realizar la predicción usando el modelo mejorado
    pred = model_mejorado.predict(img_array)
    pred_digit = int(np.argmax(pred, axis=1)[0])
    print(f"El modelo predice: {pred_digit}")